<a href="https://colab.research.google.com/github/fralfaro/MAT281/blob/main/docs/labs/lab_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# MAT281 - Laboratorio N°08

**Objetivo**: Aplicar técnicas de **machine learning no supervisado** para explorar, procesar y analizar conjuntos de datos con variables numéricas y categóricas.

> **Nota**: Puede ayudarse de algún asistente virtual como **ChatGPT, Gemini** u otros, así como del autocompletado de **Google Colab**, para avanzar en este laboratorio debido a su extensión.


## Clustering


<img src="https://www.svgrepo.com/show/253022/car.svg" width = "300" align="center"/>



El conjunto de datos **`vehiculos_procesado_con_grupos.csv`** recopila información sobre diversas características relevantes de distintos vehículos. El propósito de este ejercicio es **clasificar los vehículos en diferentes categorías**, utilizando como base las variables descritas en la tabla de atributos.

El análisis presenta un desafío adicional debido a la **naturaleza mixta de los datos**: se incluyen tanto variables **numéricas** (ej. dimensiones, consumo, emisiones) como **categóricas** (ej. tipo de tracción, tipo de combustible), lo que requiere aplicar técnicas de preprocesamiento adecuadas antes de entrenar los modelos.

Como primer paso, procederemos a **cargar y explorar el conjunto de datos**, con el fin de familiarizarnos con su estructura y las características que servirán como base para la posterior clasificación.




**Descripción de los Datos:**

| **Nombre de la Columna**   | **Descripción**                                                                                                                                   |
|----------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------|
| **year**                   | El año en que el vehículo fue fabricado.                                                                                                          |
| **desplazamiento**          | La capacidad volumétrica del motor en litros. Indica la cantidad de aire y combustible que puede desplazar el motor durante una revolución.       |
| **cilindros**               | El número de cilindros que tiene el motor. Los cilindros son las cámaras donde ocurre la combustión interna en los motores de los vehículos.       |
| **co2**                     | Emisiones de dióxido de carbono del vehículo, medido en gramos por kilómetro. Es una medida de las emisiones de gases de efecto invernadero.       |
| **clase_tipo**              | La clase o tipo de vehículo, como vehículos especiales, deportivos, etc.                                                                         |
| **traccion_tipo**           | Tipo de tracción del vehículo, ya sea tracción en dos ruedas, en cuatro ruedas o en todas las ruedas.                                             |
| **transmision_tipo**        | Tipo de transmisión del vehículo, como automática, manual, entre otros.                                                                          |
| **combustible_tipo**        | Tipo de combustible que utiliza el vehículo, como gasolina, diésel, eléctrico, híbrido, etc.                                                     |
| **tamano_motor_tipo**       | Clasificación del tamaño del motor (por ejemplo, pequeño, mediano o grande), que generalmente se basa en la capacidad de desplazamiento.           |
| **consumo_tipo**            | Clasificación del nivel de consumo de combustible del vehículo, indicando si es alto, bajo, o muy alto.                                           |
| **co2_tipo**                | Clasificación de las emisiones de CO2 del vehículo, indicando si es alto, bajo, o muy alto.                                                       |
| **consumo_litros_milla**    | El consumo de combustible del vehículo, medido en litros por milla. Indica la eficiencia del vehículo en términos de consumo de combustible.        |



In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.dummy import DummyClassifier
from sklearn.cluster import KMeans


%matplotlib inline

sns.set_palette("deep", desat=.6)
sns.set(rc={'figure.figsize':(11.7,8.27)})

In [8]:
# cargar datos
df = pd.read_csv("https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/vehiculos_procesado_con_grupos.csv", sep=",")\
       .drop(
            ["fabricante",
             "modelo",
             "transmision",
             "traccion",
             "clase",
             "combustible",
             "consumo"],

          axis=1)

df.head()

,year,desplazamiento,cilindros,co2,clase_tipo,traccion_tipo,transmision_tipo,combustible_tipo,tamano_motor_tipo,consumo_tipo,co2_tipo,consumo_litros_milla
0,1984,2.5,4.0,522.764706,Vehículos Especiales,dos,Automatica,Normal,pequeño,alto,alto,0.222671
1,1984,4.2,6.0,683.615385,Vehículos Especiales,dos,Automatica,Normal,grande,muy alto,muy alto,0.291185
2,1985,2.5,4.0,555.437500,Vehículos Especiales,dos,Automatica,Normal,pequeño,alto,alto,0.236588
3,1985,4.2,6.0,683.615385,Vehículos Especiales,dos,Automatica,Normal,grande,muy alto,muy alto,0.291185
4,1987,3.8,6.0,555.437500,Coches Medianos,dos,Automatica,Premium,grande,alto,alto,0.236588


En este caso, no solo se tienen datos numéricos, sino que también categóricos. Además, tenemos problemas de datos **vacíos (Nan)**. Así que para resolver este problema, seguiremos varios pasos:

### 1.- Normalizar datos

- Cree un conjunto de datos con las variables numéricas, además, para cada dato vacía, rellene con el promedio asociado a esa columna. Finalmente, normalize los datos mediante el procesamiento **MinMaxScaler** de **sklearn**.
- Cree un conjunto de datos con las variables categóricas , además, transforme de variables categoricas a numericas ocupando el comando **get_dummies** de pandas ([referencia](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html)). Explique a grande rasgo como se realiza la codificación de variables numéricas a categóricas.

- Junte ambos dataset en uno, llamado **df_procesado**.

In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
num_cols=df.select_dtypes(include="number").columns
cat_cols =df.select_dtypes(exclude="number").columns
df_num_scaled =pd.DataFrame(MinMaxScaler().fit_transform(df[num_cols].fillna(df[num_cols].mean() )),columns=num_cols,index=df.index)
df_cat_encoded= pd.get_dummies(df[cat_cols], dummy_na=True)
df_procesado =pd.concat([df_num_scaled, df_cat_encoded], axis=1)
print(f"Columnas numericas:{len(num_cols)} | Categoricas: {len(cat_cols)} | forma: {df_procesado.shape}")
df_procesado.head(3)

Columnas numericas:5 | Categoricas: 7 | forma: (36791, 43)


,year,desplazamiento,cilindros,co2,consumo_litros_milla,clase_tipo_Camionetas,clase_tipo_Coche Familiar,clase_tipo_Coches Grandes,clase_tipo_Coches Medianos,clase_tipo_Coches pequeños,...,consumo_tipo_moderado,consumo_tipo_muy alto,consumo_tipo_muy bajo,consumo_tipo_nan,co2_tipo_alto,co2_tipo_bajo,co2_tipo_moderado,co2_tipo_muy alto,co2_tipo_muy bajo,co2_tipo_nan
0,0.000000,0.243590,0.142857,0.398014,0.331027,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
1,0.000000,0.461538,0.285714,0.527672,0.475113,False,False,False,False,False,...,False,True,False,False,False,False,False,True,False,False
2,0.029412,0.243590,0.142857,0.424351,0.360294,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False


### 2.- Realizar ajuste mediante kmeans

Una vez depurado el conjunto de datos, es momento de aplicar el algoritmo de **kmeans**.

1. Ajuste el modelo de **kmeans** sobre el conjunto de datos, con un total de **8 clusters**.
2. Asociar a cada individuo el correspondiente cluster y calcular valor de los centroides de cada cluster.
3. Realizar un resumen de las principales cualidades de cada cluster. Para  esto debe calcular (para cluster) las siguientes medidas de resumen:
    * Valor promedio de las variables numérica
    * Moda para las variables numericas

In [10]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
kmeans =KMeans(n_clusters=8, random_state=42, n_init=10).fit(df_procesado)
labels=kmeans.labels_
df_result =df.copy()
df_result["cluster"]= labels
centroides =pd.DataFrame(kmeans.cluster_centers_,columns=df_procesado.columns)
centroides.index.name= "cluster"
num_cols =df.select_dtypes(include="number").columns.tolist()
resumen_mean_num= (df_result.groupby("cluster")[num_cols].mean(numeric_only=True).sort_index())
resumen_mode_num=(df_result.groupby("cluster")[num_cols].agg(lambda s: s.mode(dropna=True).iloc[0] if not s.mode(dropna=True).empty else np.nan).sort_index())
print("Tamanio de clusters:",df_result["cluster"].value_counts().sort_index(), "\n")
print("centroides:",centroides, "\n")
print("promedio de numericas por cluster:",resumen_mean_num, "\n")
print("Moda de numericas por cluster:", resumen_mode_num, "\n")

Tamanio de clusters: cluster
0    6057
1    7127
2    4811
3    3589
4    3987
5    3843
6    4690
7    2687
Name: count, dtype: int64 

centroides:              year  desplazamiento  cilindros       co2  consumo_litros_milla  \
cluster                                                                        
0        0.407520        0.331445   0.256838  0.354296              0.282160   
1        0.412095        0.576135   0.420153  0.506906              0.449646   
2        0.577350        0.270715   0.210649  0.305306              0.226827   
3        0.438505        0.159462   0.141141  0.242197              0.156456   
4        0.588730        0.362290   0.299545  0.354136              0.282247   
5        0.694785        0.179180   0.150608  0.240036              0.156677   
6        0.478459        0.451028   0.327460  0.410095              0.343319   
7        0.426958        0.223823   0.173660  0.304619              0.226436   

         clase_tipo_Camionetas  clase_tipo_Coche F

### 3.- Elegir Número de cluster

Estime mediante la **regla del codo**, el número de cluster apropiados para el caso.
Para efectos prácticos, eliga la siguiente secuencia como número de clusters a comparar:

$$[5, 10, 20, 30, 50, 75, 100, 200, 300]$$

Una vez realizado el gráfico, saque sus propias conclusiones del caso.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import time
X =df_procesado.values
n = X.shape[0]
k_list = [5,10,20,30,50,75,100,200,300]
k_list = [k for k in k_list if k <= n]
if not k_list:
    raise ValueError("Ningun k es valido(k>n_muestras)")
inertias, tiempos = [], []
for k in k_list:
    t0 = time.time()
    KMeans(n_clusters=k, random_state=42, n_init=20, max_iter=300).fit(X)
    inertias.append(float(_.inertia_) if (_:=KMeans(n_clusters=k, random_state=42, n_init=20, max_iter=300).fit(X)) else np.nan)  # seguridad
    tiempos.append(time.time() - t0)
inertias = np.array(inertias)
x1,y1 = k_list[0], inertias[0]
x2,y2 = k_list[-1], inertias[-1]
den = np.hypot(y2 - y1, x2 - x1)
dist = np.zeros_like(inertias) if den == 0 else np.abs((y2 - y1)*np.array(k_list) - (x2 - x1)*inertias + x2*y1 - y2*x1)/den
idx = int(np.argmax(dist))
k_rec = k_list[idx]
mejora = [np.nan] + [ (inertias[i-1]-inertias[i])/inertias[i-1] if inertias[i-1]>0 else np.nan
for i in range(1,len(inertias)) ]
plt.figure(figsize=(7,5))
plt.plot(k_list, inertias, marker='o')
plt.scatter([k_rec],[inertias[idx]], s=90)
plt.title('regla del codo-KMeans')
plt.xlabel('Numero de clusters(k)')
plt.ylabel('inercia(WcSS)')
plt.grid(True)
plt.show()
resumen = pd.DataFrame({'k':k_list,'inercia':inertias,'mejora_relativa':mejora,'dist_a_cuerda':dist,})
print(resumen.to_string(index=False))
print("Sugerencia(máxima curvatura):k =", k_rec)

Usando el metodo del codo, parece que k=75 es el punto dulce. Es donde logramos un buen balance entre que los grupos sean lo más compactos posible (reducir el WCSS) y que el modelo no sea demasiado complicado. Más allá de 75, las ganancias que obtenemos por añadir un cluster más son mínimas; el WCSS apenas mejora, por eso hablamos de rendimientos decrecientes. Así que, si quieres la opción más sensata, ve por k=75. Si prefieres que sea más fácil de entender, k=50 es una alternativa más limpia, y si necesitas ir al máximo detalle, podrías estirarte hasta k=100

Al observar el gráfico resultante, se pueden obtener conclusiones sobre el número apropiado de clusters. La regla del codo sugiere elegir el número de clusters donde la reducción en la inercia se estabiliza significativamente. En otras palabras, se busca el punto en el gráfico donde la curva de inercia comienza a aplanarse o forma un codo.

## Reducción de Dimensionalidad

<img src="https://1000logos.net/wp-content/uploads/2020/11/Wine-Logo-old.png" width = "300" align="center"/>


Para este ejercicio utilizaremos el **Wine Dataset**, un conjunto de datos clásico disponible en la librería **scikit-learn** y en el repositorio de la **UCI Machine Learning**.
Este dataset contiene información de **178 muestras de vino** provenientes de la región italiana de *Piamonte*. Cada vino pertenece a una de **tres variedades de uva** (*clases*), que actúan como etiquetas para el análisis supervisado, pero aquí se usarán solo como referencia en la visualización.

Cada muestra está descrita por **13 variables químicas** obtenidas de un análisis de laboratorio, entre ellas:

* **Alcohol**: porcentaje de alcohol en el vino.
* **Malic acid**: concentración de ácido málico.
* **Ash**: contenido de ceniza.
* **Alcalinity of ash**: alcalinidad de la ceniza.
* **Magnesium**: cantidad de magnesio (mg/L).
* **Total phenols**: concentración total de fenoles.
* **Flavanoids**: tipo de fenoles con propiedades antioxidantes.
* **Nonflavanoid phenols**: fenoles que no son flavonoides.
* **Proanthocyanins**: compuestos relacionados con el color y el sabor.
* **Color intensity**: intensidad del color del vino.
* **Hue**: matiz del color.
* **OD280/OD315 of diluted wines**: relación de absorbancia que mide la calidad del vino.
* **Proline**: concentración de prolina (un aminoácido).

Estas características permiten representar cada vino como un punto en un espacio de **13 dimensiones**.

El objetivo del análisis con este dataset es **reducir la dimensionalidad** para visualizar y explorar patrones en los datos. Para ello aplicaremos:

* **PCA (Principal Component Analysis):** identificar las combinaciones lineales de variables que explican la mayor varianza en el conjunto.
* **t-SNE (t-distributed Stochastic Neighbor Embedding):** mapear las muestras a 2D o 3D, preservando relaciones de vecindad y estructuras no lineales.

La comparación entre ambas técnicas permitirá observar cómo las tres clases de vinos se diferencian en el espacio reducido y discutir la utilidad de la reducción de dimensionalidad en datos con mayor número de variables que en el caso del dataset *Wine*.



In [ ]:
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

In [ ]:
# cargar dataset
dataset = load_wine()

# nombres de las variables
features = dataset.feature_names
target = 'wine_class'

# construir DataFrame
wine = pd.DataFrame(dataset.data, columns=features)
wine[target] = dataset.target

# ver primeras filas
wine.head()



### 1. **Análisis detallado con PCA**

* Calcular la **varianza explicada** por cada componente principal y representar el gráfico de varianza acumulada, identificando cuántos componentes son necesarios para capturar al menos el **90–95% de la información**.
* Construir tablas y gráficos que muestren cómo las observaciones (vinos) se proyectan en las primeras componentes principales.
* Analizar los **loadings** (coeficientes de cada variable en los componentes) e interpretar qué características químicas del vino (alcohol, fenoles, color, etc.) tienen mayor influencia en las nuevas dimensiones.
* Visualizar los datos reducidos a 2D o 3D e interpretar si las **tres variedades de vino** se separan de forma clara en el espacio proyectado.



In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
features = [c for c in wine.columns if c != 'wine_class']
X_std = StandardScaler().fit_transform(wine[features].values)
n_comp = min(30, len(features))
pca = PCA(n_components=n_comp, svd_solver="full")
Z = pca.fit_transform(X_std)
exp_var = pca.explained_variance_ratio_
cum_var = np.cumsum(exp_var)
n90 = int(np.argmax(cum_var >= 0.90) + 1) if (cum_var >= 0.90).any() else n_comp
n95 = int(np.argmax(cum_var >= 0.95) + 1) if (cum_var >= 0.95).any() else n_comp
print(pd.DataFrame({"PC": np.arange(1, n_comp+1), "var_exp": exp_var, "var_acum": cum_var}).to_string(index=False))
print(f"Componentes para>=90%:{n90}")
print(f"Componentes para >=95%: {n95}")
pcs = np.arange(1, n_comp+1)
plt.figure(figsize=(7,5))
plt.plot(pcs, exp_var, marker='o',label='Varianza explicada (PC_i)')
plt.plot(pcs, cum_var, marker='s', label='Varianza acumulada')
plt.axhline(0.90, linestyle='--'); plt.axhline(0.95, linestyle='--')
plt.axvline(n90, linestyle=':', alpha=0.7); plt.axvline(n95, linestyle=':', alpha=0.7)
plt.xlabel('Cmponente principal'); plt.ylabel('Proporcion de varianza')
plt.title('PCA: Scree y varianza acumulada'); plt.grid(True); plt.legend(); plt.show()
k = min(3, Z.shape[1])
df_pca = pd.DataFrame(Z[:, :k], columns=[f'PC{i}' for i in range(1, k+1)], index=wine.index)
print("Proyecciones (primeras filas) ")
print(df_pca.head(10).to_string())
labels =wine['wine_class']
names= dataset.target_names if 'dataset' in globals() and hasattr(dataset, 'target_names') else {c: f"class {c}" for c in sorted(labels.unique())}
if isinstance(names,(list, np.ndarray)):
    names= {i: names[i] for i in range(len(names))}
labels_str=labels.map(names)
plt.figure(figsize=(7,6))
xlab =f"PC1 ({exp_var[0]:.1%})"; ylab = f"PC2 ({exp_var[1]:.1%})"
for g,sub in df_pca.groupby(labels_str): plt.scatter(sub['PC1'], sub['PC2'], s=25, label=str(g))
plt.legend(title='Variedad'); plt.xlabel(xlab); plt.ylabel(ylab)
plt.title('Proyección PC1–PC2'); plt.grid(True); plt.show()
if k >= 3:
    from mpl_toolkits.mplot3d import Axes3D
    fig = plt.figure(figsize=(8,6)); ax = fig.add_subplot(111, projection='3d')
    for g, sub in df_pca.groupby(labels_str):
      ax.scatter(sub['PC1'],sub['PC2'], sub['PC3'], s=20, label=str(g))
    ax.legend(title='Variedad'); ax.set_xlabel(f"PC1 ({exp_var[0]:.1%})")
    ax.set_ylabel(f"PC2 ({exp_var[1]:.1%})"); ax.set_zlabel(f"PC3 ({exp_var[2]:.1%})")
    ax.set_title('Proyección PC1–PC2–PC3'); plt.show()
loadings = pd.DataFrame(pca.components_.T, index=features, columns=[f'PC{i}' for i in range(1, n_comp+1)])
for pc in [f'PC{i}' for i in range(1, min(3, n_comp)+1)]:
    idx = loadings[pc].abs().nlargest(10).index
    print(f"Top 10 en {pc}:")
    print(loadings.loc[idx, pc].sort_values(key=np.abs, ascending=False).to_string())



### 2. **Análisis detallado con t-SNE**

* Aplicar **t-SNE** para reducir los datos a 2 dimensiones, probando diferentes configuraciones de hiperparámetros como *perplexity* y *learning rate*.
* Comparar las distintas visualizaciones obtenidas y discutir cómo los hiperparámetros afectan la estructura de los clústeres.
* Analizar si las **tres clases de vinos** forman agrupaciones definidas y si t-SNE logra capturar relaciones no lineales que PCA no refleja.



In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
X=df_procesado.values
y =None
df_lbl =df if 'df'in globals() else (wine if 'wine' in globals() else None)
if df_lbl is not None:
    for c in ['wine_class','target','class','Type','variedad','variety','cluster']:
        if c in df_lbl.columns:
            y = df_lbl[c].values
            break
Xd =PCA(n_components=min(30, X.shape[1]), random_state=42).fit_transform(X)
perps, lrs = [5, 30], [200]
rs = np.random.RandomState(42)
idx = rs.choice(len(Xd), size=min(len(Xd), 1500), replace=False)
Xs = Xd[idx]; ys = None if y is None else y[idx]
Z={}
for p in perps:
    for lr in lrs:
        Z[(p,lr)] = TSNE(n_components=2, perplexity=p, learning_rate=lr,
                         n_iter=500, init='pca', angle=0.8, random_state=42).fit_transform(Xs)
fig, axs =plt.subplots(len(perps), len(lrs), figsize=(6*len(lrs), 3.2*len(perps)), squeeze=False)
for i,p in enumerate(perps):
    for j,lr in enumerate(lrs):
        T, ax = Z[(p,lr)],axs[i,j]
        if ys is None:
            ax.scatter(T[:,0], T[:,1], s=7)
        else:
            yser=pd.Series(ys)
            for g, ids in yser.groupby(yser).groups.items():
                ax.scatter(T[list(ids),0], T[list(ids),1], s=7, label=str(g))
            if i==0 and j==0: ax.legend(title="Clase", fontsize=8)
        ax.set_title(f"perp={p}, lr={lr}"); ax.grid(True)
plt.tight_layout(); plt.show()
p0,lr0 =30, 200
Z_tsne= TSNE(n_components=2, perplexity=p0, learning_rate=lr0,n_iter=750, init='pca', angle=0.8,random_state=42).fit_transform(Xd)
Z_pca2 =PCA(n_components=2, random_state=42).fit_transform(X)
fig,ax =plt.subplots(1,2, figsize=(10,4.2))
for k,(M,tit)in enumerate([(Z_pca2,"PCA (2D)"), (Z_tsne, f"t-SNE (perp={p0}, lr={lr0})")]):
    if y is None:
        ax[k].scatter(M[:,0], M[:,1], s=7)
    else:
        yser = pd.Series(y)
        for g, ids in yser.groupby(yser).groups.items():
            ax[k].scatter(M[list(ids),0], M[list(ids),1], s=7, label=str(g))
        if k==0:
          ax[k].legend(title="Clase", fontsize=8)
    ax[k].set_title(tit); ax[k].grid(True)
plt.tight_layout(); plt.show()



### 3. **Comparación entre PCA y t-SNE**

* Contrastar las visualizaciones y discutir las **ventajas y limitaciones** de cada técnica:

  * PCA como método **lineal** para interpretar varianza y relaciones globales.
  * t-SNE como método **no lineal** que preserva relaciones locales y vecindades.
* Evaluar en qué escenarios prácticos sería más recomendable usar PCA (interpretabilidad, reducción previa para modelos) o t-SNE (exploración y visualización de clústeres).
* Reflexionar sobre la **importancia de la reducción de dimensionalidad** en datasets de alta dimensión como Wine, destacando su utilidad para:

  * Visualizar patrones ocultos en los datos.
  * Reducir complejidad y ruido antes de aplicar algoritmos de aprendizaje automático.
  * Facilitar la interpretación y comunicación de resultados.



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE, trustworthiness
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_wine
wine =load_wine()
df_procesado =pd.DataFrame(wine.data, columns=wine.feature_names)
df_procesado['variedad'] = wine.target
X=df_procesado.select_dtypes(include=[np.number]).values
y_cols= ('variedad','variety','target','class','Type','WineType','cluster')
y= None
for c in y_cols:
    if c in df_procesado.columns:
      y=df_procesado[c].to_numpy()
      break
have_y, cv =False, None
if y is not None:
    y = pd.Series(y)
    mask = y.notna().values
    have_y = (mask.sum() >= 10) and (y[mask].nunique() >= 2)
    if have_y: cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
pca =PCA(2).fit(X)
Zp = pca.transform(X)
exp2 =pca.explained_variance_ratio_.sum()
tw_p=trustworthiness(X, Zp,n_neighbors=5)
knn_p = np.nan if not have_y else cross_val_score(KNeighborsClassifier(5), Zp[mask], y[mask], cv=cv).mean()
perps = [5, 30, 50]
lrs = [200, 1000]
best,Zt, tw_t, knn_t, p_best,lr_best=(-np.inf, None, np.nan, np.nan, np.nan, np.nan)
rows =[]
for p in [p for p in perps if p < len(X)]:
    for lr in lrs:
        Z =TSNE(2,perplexity=p, learning_rate=lr, n_iter=1000, init='pca', random_state=42).fit_transform(X)
        tw =trustworthiness(X, Z, n_neighbors=5)
        knn= np.nan if not have_y else cross_val_score(KNeighborsClassifier(5), Z[mask], y[mask], cv=cv).mean()
        score =knn if have_y and not np.isnan(knn) else tw
        if score > best:
            best, Zt, tw_t, knn_t, p_best, lr_best = score, Z, tw, knn, p, lr
        rows.append({'perp': p, 'lr': lr, 'tw':tw, 'knn_acc':knn})
res =pd.DataFrame(rows).sort_values(['perp','lr'])
comp=pd.DataFrame({'embedding':['PCA_2D','tSNE_2D'],'var_exp_2D':[exp2, np.nan],'trustworthiness':[tw_p, tw_t],'knn_acc_cv5':[knn_p, knn_t]})
print(f"Mejort-SNE:Perplexity={p_best}, Learning Rate={lr_best}")
print("="*50)
print("a) Optimización t-SNE (Métricas)")
print(res.to_string(index=False))
print("b) Comparación PCA vs. t-SNE (Métricas Clave)")
print(comp.to_string(index=False))
plt.figure(figsize=(12, 5))
titles = [f'PCA 2D (Var. Expl.={exp2:.3f})', f't-SNE 2D (perp={p_best}, lr={lr_best})']
Zs = [Zp, Zt]
for i, Z in enumerate(Zs):
    ax = plt.subplot(1, 2, i + 1)
    if have_y:
        for g in np.unique(y[mask]):
            idx = (y ==g).values
            ax.scatter(Z[idx,0],Z[idx, 1], s=12, label=str(g))
        ax.legend(title="Etiqueta", fontsize=8)
    else:
        ax.scatter(Z[:, 0], Z[:, 1], s=12)
    ax.set(title=titles[i], xlabel='Dim 1' if i==1 else 'PC1', ylabel='Dim 2' if i == 1 else 'PC2')
    ax.grid(True)
plt.tight_layout()
plt.show()

la reduccion de dimensionalidad es fundamental cuando trabajamos con datasets grandes como el Wine, que tienen muchas caracteristicas (13 en este caso). Basicamente, nos ayuda a ver lo invisible: al pasar de 13D a solo 2D, podemos visualizar patrones ocultos como la separacion clara de las variedades de vino en el grafico t-SNE algo imposible de percibir con datos crudos. Ademas, al enfocarnos solo en la informacion esencial, logramos reducir la complejidad y el ruido en el dataset, lo que a su vez hace que los algoritmos de machine learning sean mas rapidos y precisos. Al final del dia, esto facilita enormemente la interpretacion y comunicacion de resultados, ya que es mucho mas sencillo mostrar un grafico 2D que una tabla con trece columnas. Es la clave para simplificar sin perder la esencia de los datos